### Задачи к Лекции 4

__Исходные данные__ 

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv(r"C:\Users\Роксана\Downloads\mlbootcamp5_train.csv", 
                 sep=";", 
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head(1)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [11]:
from scipy.spatial.distance import cdist
from collections import Counter
from sklearn.model_selection import train_test_split

In [21]:
#выбираю признаки
X = clean_df[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo']].values
y = clean_df['cardio'].values
#делю выборку на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
class KNNClassifier:
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors
        self.X_train = None
        self.y_train = None
    #сохраняю обучающие данные
    def fit(self, X, y): 
        self.X_train = np.array(X) #преобразую их в numpy массивы для удобства вычислений и сохраняю в атрибутах класса
        self.y_train = np.array(y)
    #предсказываю класс для новых данных
    def predict(self, X): 
        X = np.array(X)
        predictions = []
        for x in X:
            #вычисляю расстояния между текущей точкой x и всеми точками обучающей выборки
            #используется Евклидова метрика
            distances = cdist([x], self.X_train, metric='euclidean')[0]  
            #нахожу k ближайших соседей
            nearest_indices = np.argsort(distances)[:self.n_neighbors] #возвращаю индексы точек, отсортированных по расстоянию (от ближайшей к самой далёкой)
            nearest_labels = self.y_train[nearest_indices] #получаю метки этих ближайших соседей
            #выбираю наиболее частый класс
            most_common = Counter(nearest_labels).most_common(1)[0][0] #считаю кол-во каждого класса среди соседей
            predictions.append(most_common)
        return np.array(predictions)
    #вычисляю точность
    def score(self, X, y): 
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

In [22]:
#создаю классификатор с k=5 и евклидовой метрикой
knn = KNNClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
#делаю предсказание
y_pred = knn.predict(X_test)
#провожу оценку точности
accuracy = knn.score(X_test, y_test)
print(f"Точность: {accuracy:.3f}")

Точность: 0.678


**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [30]:
A = clean_df[['age', 'weight', 'height', 'ap_hi', 'ap_lo']].values
b = clean_df['cardio'].values
#делю выборку на тренировочную и тестовую
A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.3)

In [31]:
#инициализация моделей
knn = KNeighborsClassifier(n_neighbors=5)
nb = GaussianNB()
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
#обучение
knn.fit(A_train, b_train)
nb.fit(A_train, b_train)
dt.fit(A_train, b_train)

DecisionTreeClassifier(max_depth=3, random_state=42)

In [32]:
#нахожу accuracy для каждого классификатора
knn_acc = accuracy_score(b_test, knn.predict(A_test))
nb_acc = accuracy_score(b_test, nb.predict(A_test))
dt_acc = accuracy_score(b_test, dt.predict(A_test))
#нахожу F1-меру, для каждого классификатора
knn_f1 = f1_score(b_test, knn.predict(A_test))
nb_f1 = f1_score(b_test, nb.predict(A_test))
dt_f1 = f1_score(b_test, dt.predict(A_test))
#нахожу ROC AUC для каждого классификатора
knn_auc = roc_auc_score(b_test, knn.predict_proba(A_test)[:, 1])
nb_auc = roc_auc_score(b_test, nb.predict_proba(A_test)[:, 1])
dt_auc = roc_auc_score(b_test, dt.predict_proba(A_test)[:, 1])
#нахожу функцию потерь для каждого классификатора
knn_loss = log_loss(b_test, knn.predict_proba(A_test))
nb_loss = log_loss(b_test, nb.predict_proba(A_test))
dt_loss = log_loss(b_test, dt.predict_proba(A_test))

In [33]:
#делаю табличку
results = {
    'kNN': {
        'Accuracy': knn_acc,
        'F1': knn_f1,
        'ROC AUC': knn_auc,
        'Log Loss': knn_loss
    },
    'NB': {
        'Accuracy': nb_acc,
        'F1': nb_f1,
        'ROC AUC': nb_auc,
        'Log Loss': nb_loss
    },
    'DT': {
        'Accuracy': dt_acc,
        'F1': dt_f1,
        'ROC AUC': dt_auc,
        'Log Loss': dt_loss
    }
}
results_df = pd.DataFrame(results).T #транспонирую для удобства
results_df

,Accuracy,F1,ROC AUC,Log Loss
kNN,0.677806,0.666164,0.724368,2.311064
NB,0.706373,0.663517,0.776238,0.633745
DT,0.713697,0.700705,0.772191,0.566717


**Комментарии:** В итоге дерево решений справилось лучше всего